# Theft Prevention using AI

In [1]:
import numpy as np
import cv2
import os
import smtplib
import time
from scipy.spatial.distance import cosine

from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, ZeroPadding2D, MaxPooling2D, Flatten, Dropout, Activation
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras.models import model_from_json

In [2]:
sender = "ar7972278@gmail.com"
reciever = "ar7972278@gmail.com"
password = "pysmtp2278"

server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(sender, password)

(235, b'2.7.0 Accepted')

In [3]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def save_user():
    capture = cv2.VideoCapture(0)
    count = len(os.listdir("data"))
    while True:
        _, image = capture.read()
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            # the bounding box around the detected object
            cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 255), 3)
            face = cv2.resize(gray[y:y+h, x:x+w], (224, 224))
            cv2.putText(face, str(count), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        if cv2.waitKey(15)==13 :
            break
        if cv2.waitKey(1) & 0xFF == ord('s'):
            file_path = "data/"+str(count)+".jpg"
            cv2.imwrite(file_path, face)
            count+=1
        cv2.imshow('face frame', image)
    capture.release()
    cv2.destroyAllWindows()

In [4]:
save_user()

In [5]:
def compare(face1, face2):
    score = cosine(face1, face2)
    return score

In [6]:
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)

    img = preprocess_input(img)
    return img

In [7]:
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Conv2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))
model.load_weights('vgg_face_weights.h5') # Model Link: https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)


In [8]:
valids = list()
for filename in os.listdir("data"):
    valids.append(model.predict(preprocess_image("data/"+filename)))

In [9]:
cap = cv2.VideoCapture(0)
intruder_pics = list()
max_intruders = 0
last_sent = 0;
cnt = 0;
while (True):
    _, img = cap.read()
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if faces is None:
        continue
    
    intruders = 0
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
        cap_face = cv2.resize(img[y:y+h, x:x+w], (224, 224))

        img_data = image.img_to_array(cap_face)
        img_data = np.expand_dims(img_data, axis=0)/255.0

        captured_embeddings = model.predict(img_data)[0, :]

        found = 0
        for i in valids:
            embeddings = i

            score = compare(embeddings, captured_embeddings)
            if (score < 0.45):
                cv2.putText(img, "Valid", (x+w, y-12), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                found = 1
                break

        if (found == 0):
            cv2.putText(img, "Intruder", (x+w, y-12), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            intruders+=1
            intruder_pics.append(cap_face)
            message="Subject: Intruders!!!\n\nThere are people without validation near you place"
            if last_sent+300 <= time.time():
                server.sendmail(sender, reciever, message)
                last_sent = time.time()
                file_path = "intruders\\"+str(cnt)+".jpg"
                cnt+=1
                cv2.imwrite(file_path, img)
        if max_intruders<intruders:
            max_intruders = intruders
        cv2.putText(img, "Count:"+str(max_intruders), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('img', img)

    if cv2.waitKey(10) & 0xFF == 13:
        break

cap.release()
cv2.destroyAllWindows()